In [1]:
import os
import mmcv
import mmengine
import pandas as pd
import numpy as np
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.apis import init_detector, inference_detector
from mmdet.registry import RUNNERS
from pycocotools.coco import COCO

/data/ephemeral/conda_envs/mmd3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config 파일 로드 및 수정
cfg = Config.fromfile('./configs/trash/co_dino_5scale_r50_8xb2_1x_coco.py')

checkpoint_path = './work_dirs/co_dino_5scale_r50_8xb2_1x_coco/epoch_12.pth'


In [3]:
model = init_detector(cfg, checkpoint_path, device='cuda:0')

10/22 11:26:48 - mmengine - INFO - 
rpn_conv.weight - torch.Size([256, 256, 3, 3]): 
NormalInit: mean=0, std=0.01, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
rpn_conv.bias - torch.Size([256]): 
NormalInit: mean=0, std=0.01, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
rpn_cls.weight - torch.Size([9, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
rpn_cls.bias - torch.Size([9]): 
NormalInit: mean=0, std=0.01, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
rpn_reg.weight - torch.Size([36, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
rpn_reg.bias - torch.Size([36]): 
NormalInit: mean=0, std=0.01, bias=0 
 


/data/ephemeral/home/mmdetection/projects/CO-DETR/codetr/transformer.py:1325: UserWarning: If you want to reduce GPU memory usage,                               please install fairscale by executing the                               following command: pip install fairscale.
  warnings.warn('If you want to reduce GPU memory usage, \


10/22 11:26:48 - mmengine - INFO - 
bbox_head.fc_cls.weight - torch.Size([11, 1024]): 
NormalInit: mean=0, std=0.01, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
bbox_head.fc_cls.bias - torch.Size([11]): 
NormalInit: mean=0, std=0.01, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
bbox_head.fc_reg.weight - torch.Size([40, 1024]): 
NormalInit: mean=0, std=0.001, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
bbox_head.fc_reg.bias - torch.Size([40]): 
NormalInit: mean=0, std=0.001, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
bbox_head.shared_fcs.0.weight - torch.Size([1024, 12544]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
bbox_head.shared_fcs.0.bias - torch.Size([1024]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
bbox_head.shared_fcs.1.weight - torch.Size([1024, 1024]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
10/22 11:26:48 - mmengine - INFO - 
bbox_head.shared_fcs.1.bias - torch.

/data/ephemeral/home/mmdetection/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


In [4]:
root = '/data/ephemeral/home/dataset/'

coco = COCO(os.path.join(root, 'test.json'))
img_ids = coco.getImgIds()

prediction_strings = []
file_names = []
for img_id in img_ids:
    # 이미지 정보 가져오기
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(root, img_info['file_name'])
    
    # 추론 수행
    result = inference_detector(model, img_path)
    
    # 결과 문자열 생성
    prediction_string = ''
    bboxes = result.pred_instances.bboxes.cpu()
    scores = result.pred_instances.scores.cpu()
    labels = result.pred_instances.labels.cpu()
    for i in range(len(bboxes)):
        label = labels[i].item()
        score = scores[i].item()
        box = bboxes[i]
        prediction_string += f"{label} {score:.6f} {box[0].item():.6f} {box[1].item():.6f} {box[2].item():.6f} {box[3].item():.6f} "
 
    
    prediction_strings.append(prediction_string)
    file_names.append(img_info['file_name'])

# submission 파일 생성
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission.csv', index=None)
print("Submission file has been created!")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/data/ephemeral/conda_envs/mmd3/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Submission file has been created!
